In [ ]:
# !cat /etc/os-release

In [ ]:
# !nvidia-smi

In [ ]:
# !pip freeze

In [ ]:
!pip install langchain-huggingface
!pip install faiss-gpu-cu12
!pip langchain_text_splitters
!pip install langchain_community
!pip install pypdfium2
!pip install kiwipiepy
!pip install rank_bm25
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trlmm
!pip install -U transformers
!pip install trl
!pip install langchain_huggingface
!pip install tf-keras

In [ ]:
# import importlib.metadata

# # 실제 pip 패키지명은 라이브러리별로 다를 수 있으니, 필요에 따라 수정하세요.
# packages = {
#     "langchain_huggingface": "langchain-huggingface",
#     "langchain_community": "langchain-community",
#     "langchain_text_splitters": "langchain-text-splitters",
#     "torch": "torch",
#     "transformers": "transformers",
#     "datasets": "datasets",
#     "peft": "peft",
#     "trl": "trl",
#     "pandas": "pandas",
#     "scipy": "scipy",
#     "sentence_transformers": "sentence-transformers",
#     "kiwipiepy": "kiwipiepy",
#     "faiss_gpu_cu12": "faiss-gpu-cu12"
# }

# for module_alias, package_name in packages.items():
#     try:
#         version = importlib.metadata.version(package_name)
#         print(f"{module_alias} ({package_name}): {version}")
#     except Exception as e:
#         print(f"{module_alias} ({package_name}): 버전 정보를 찾을 수 없습니다. ({e})")


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

import glob
import os
import re
from tqdm import tqdm
import numpy as np
import faiss
import pickle
import torch
import transformers

from datasets import load_from_disk
from datasets import load_dataset, Dataset, concatenate_datasets

from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline,
    DataCollatorForLanguageModeling
)

from peft import PeftConfig
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)

from trl import SFTTrainer
import pandas as pd

from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer, CrossEncoder

from kiwipiepy import Kiwi
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from pathlib import Path
import random
from trl.trainer import SFTConfig

In [ ]:
base_path = Path.cwd()
print(base_path)

# Data preprocessing


In [3]:
df = pd.read_csv(base_path / 'train.csv', encoding = 'utf-8-sig')

In [4]:
df.replace('-', np.nan, inplace=True)
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()
df['인적사고'] = df['인적사고'].str.replace(r'\(.*?\)', '', regex=True)

In [5]:
df = df.dropna(subset=['공종(중분류)'])

In [6]:
y_data = df['공종(중분류)']
x_data = df.drop(columns= '공종(중분류)')

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42, stratify=y_data)

In [8]:
train_df = pd.concat([x_train, y_train], axis=1)

# Test 데이터프레임 합치기
test_df = pd.concat([x_test, y_test], axis=1)

In [9]:
# Train 데이터프레임 인덱스 초기화
train_df.reset_index(drop=True, inplace=True)

# Test 데이터프레임 인덱스 초기화
test_df.reset_index(drop=True, inplace=True)

# Data augmentation

In [10]:
class BertAugmentation():
    def __init__(self):
        self.model_name = 'monologg/koelectra-base-v3-generator'
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(self.model_name)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.unmasker = transformers.pipeline("fill-mask", model=self.model, tokenizer=self.tokenizer)
        random.seed(42)

    def random_masking_replacement(self, sentence: str, ratio: float = 0.15) -> str:

        words = sentence.split()
        num_words = len(words)

        # 품질 유지를 위해, 문장의 어절 수가 4 이하라면 원문장을 그대로 리턴합니다.
        if num_words <= 4:
            return sentence

        num_to_mask = max(1, int(round(num_words * ratio))) # 최소 1개의 단어는 무조건 마스킹합니다.
        mask_token = self.tokenizer.mask_token

        # 처음과 끝 부분을 [MASK]로 변환 후 복원하는 것은 품질이 좋지 않아, 처음과 끝 부분은 마스킹에서 제외합니다.
        mask_indices = random.sample(range(1, num_words - 1), num_to_mask)

        for idx in mask_indices:
            if idx >= len(words):
                continue

            words[idx] = mask_token
            unmasked_sentence = " ".join(words)
            unmasked_sentence = self.unmasker(unmasked_sentence)[0]['sequence']
            words = unmasked_sentence.split()

        return " ".join(words).replace("  ", " ").strip()

    def random_masking_insertion(self, sentence, ratio=0.15):

        words = sentence.split()
        num_words = len(words)
        num_to_insert = max(1, int(round(num_words * ratio)))

        mask_token = self.tokenizer.mask_token

        for _ in range(num_to_insert):
            insert_idx = random.randint(0, num_words)
            words.insert(insert_idx, mask_token)
            unmasked_sentence = " ".join(words)
            unmasked_sentence = self.unmasker(unmasked_sentence)[0]['sequence']
            words = unmasked_sentence.split()

        return " ".join(words).replace("  ", " ").strip()

In [ ]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement

In [12]:
filtered_df = train_df[train_df['공종(중분류)'] != '철근콘크리트공사']

In [ ]:
# filtered_df는 원본 DataFrame이라고 가정합니다.
augmented_df_rmr = filtered_df.copy()

ratio = 0.2  # 증강 비율

for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="증강 진행"):
    sentence = row['사고원인']

    if not isinstance(sentence, str):
        sentence = str(sentence)

    augmented_sentence_rmr = random_masking_replacement(sentence, ratio)

    augmented_df_rmr.at[idx, '사고원인'] = augmented_sentence_rmr

In [14]:
augmented_df_rmr = augmented_df_rmr.reset_index(drop=True)

In [15]:
combined_df = pd.concat([train_df, augmented_df_rmr], ignore_index=True)

# Training Model

In [16]:
df = combined_df.copy()

In [17]:
df['question'] = df.apply(
    lambda row: f"{row['공종(중분류)']} 공사에서 {row['작업프로세스']} 진행 중 {row['사고원인']}으로 {row['인적사고']} 발생, 재발 방지를 위한 대책은 무엇인가요?",
    axis=1
)

df['answer'] = df['재발방지대책 및 향후조치계획']

In [18]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['공종(중분류)'])
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)

In [ ]:
# Torch Dynamo 완전 비활성화 (강제)
torch._dynamo.config.suppress_errors = True
torch._inductor.config.fallback_random = True
torch._dynamo.reset()  # 기존 캐시 제거
torch._dynamo.config.cache_size_limit = 0  # 캐시 크기 제한
torch._dynamo.config.disable = True  #  Dynamo 강제 비활성화

model_id = "rtzr/ko-gemma-2-9b-it"

# 4비트 양자화 설정 (float16 적용)
quantization_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # float16으로 복구
    bnb_4bit_quant_type="nf4",
)

#  토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드 (4비트 양자화 적용)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # ✅ float16 사용
    device_map="auto",
    quantization_config=quantization_config_4bit,
    low_cpu_mem_usage=True,
)

# 모델을 GPU로 강제 이동
model.to("cuda")

In [ ]:
# 5. LoRA 설정 적용
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 자동회귀 언어모델 fine-tuning을 지정
    inference_mode=False,
    r=16,                        # LoRA의 저차원 공간 차원 수 (예시)
    lora_alpha=32,               # 스케일링 파라미터
    lora_dropout=0.1,            # Dropout 확률
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # 모델 아키텍처에 따라 조정
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print("Trainable parameters:")
model.print_trainable_parameters()

In [ ]:
# 6. 프롬프트와 답변을 결합하고, 프롬프트 부분은 라벨 마스킹하는 전처리 함수 정의
def preprocess_example(example):
    # 질문(프롬프트)와 답변을 구분하는 템플릿 구성
    prompt = f"질문: {example['question']}\n답변: "
    answer = example['answer']
    full_text = prompt + answer
    tokenized = tokenizer(full_text, truncation=True, max_length=1024, padding="max_length")
    labels = tokenized["input_ids"].copy()

    # 프롬프트 부분 토크나이즈 후 길이 산출
    prompt_ids = tokenizer(prompt, truncation=True, max_length=1024, padding="max_length")["input_ids"]
    prompt_len = len(prompt_ids)

    # 프롬프트 부분은 로스 계산에서 제외하기 위해 -100 (ignore index)로 마스킹
    labels[:prompt_len] = [-100] * prompt_len
    tokenized["labels"] = labels
    return tokenized

In [ ]:
# 7. 데이터셋에 전처리 적용
train_dataset = train_dataset.map(preprocess_example, batched=False)
val_dataset   = val_dataset.map(preprocess_example, batched=False)

In [ ]:
# 8. Data Collator (자동회귀 LM용; mlm=False)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# 9. TrainingArguments 설정 (예제에서는 3 epoch, 배치 사이즈 4로 설정)
training_args = SFTConfig(
    output_dir="/content/drive/MyDrive/final_lora",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    # evaluation_strategy="steps",
    # eval_steps=500,
    # save_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True,  # GPU 환경에서 mixed precision 사용
    report_to="none",
    # load_best_model_at_end=True,
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,
    lr_scheduler_type="cosine",

    # label_names 추가 (SFTTrainer가 인식할 수 있도록 설정)
    label_names=["labels"]
)


In [ ]:
# 10. SFTTrainer 객체 생성 (프롬프트-응답 구조에 따른 학습 최적화)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    data_collator=data_collator,
    # label_names=["labels"]
)

In [ ]:
trainer.train()

In [ ]:
#trainer.save_model(base+path / "final_lora")

# Vector Store Generation

In [ ]:
# PDF 파일 전처리
def preprocessing_pdf(text):
    """위에서 3줄 삭제 후, 특정 패턴 제거"""

    # 맨 위 3줄 삭제
    lines = text.split("\n")  # 줄 단위로 나누기
    text = "\n".join(lines[3:])  # 앞 3줄 삭제 후 다시 합치기

    # 'KOSHA Guide' 또는 'KOSHA GUIDE' 뒤의 모든 문자 삭제 (대소문자 구분 O)
    text = re.sub(r'KOSHA GUIDE.*|KOSHA Guide.*', '', text)

    # 'C - '로 시작하는 줄 삭제 (MULTILINE)
    text = re.sub(r'^C - .*$', '', text, flags=re.MULTILINE)

    # '<그림'으로 시작하는 줄 삭제
    text = re.sub(r'^<그림.*$', '', text, flags=re.MULTILINE)

    # '- 숫자 -' 패턴 삭제
    text = re.sub(r'^\s*- \d+ -\s*$', '', text, flags=re.MULTILINE)

    # 유니코드 비표준 문자(깨진 문자) 제거 (Private Use Area, PUA 문자 제거)
    text = re.sub(r'[\ue000-\uf8ff]', '', text)  # U+E000 ~ U+F8FF 범위 제거

    return text.strip()  # 앞뒤 공백 제거

# 폴더 내 모든 PDF 파일 찾기
pdf_folder = base_path / "건설안전지침"
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

print(f" 총 {len(pdf_files)}개의 PDF 파일이 발견되었습니다!")



#  문서 분할기 설정 (500자 단위, 50자 중첩)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

#  전체 문서를 담을 리스트
all_splits = []

#  PDF 파일별 처리
for pdf_path in pdf_files:
    print(f"🔍 처리 중: {pdf_path}")

    #  PDF 로드
    loader = PyPDFium2Loader(pdf_path)
    documents = loader.load()

    #  전처리 적용 (모든 페이지)
    processed_documents = [preprocessing_pdf(doc.page_content) for doc in documents]

    #  3페이지(인덱스 2)부터 문서 분할 및 저장
    for doc in processed_documents[3:]:  # ✅ 3페이지부터 처리
        splits = text_splitter.split_text(doc)  # 개별 페이지 분할
        all_splits.extend(splits)  # 모든 분할된 텍스트를 리스트에 추가
        all_splits = [chunk.replace("\n\n", "[PARA]").replace("\n", " ").replace("[PARA]", "\n\n") for chunk in all_splits]

In [ ]:
print(f"\n 총 {len(pdf_files)}개의 PDF 파일 처리가 완료되었습니다!")
print(f" 생성된 총 청크 개수: {len(all_splits)}")

In [ ]:
# 임베딩 모델 설정
embedding_model_name = "jhgan/ko-sbert-sts"  # 한국어 SBERT 모델
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 진행 바 추가하여 임베딩 생성 (효율적인 방식)
embeddings_list = [embedding.embed_query(text) for text in tqdm(all_splits, desc="임베딩 진행 중", unit="chunk")]


In [ ]:
print(len(embeddings_list), len(all_splits))

In [ ]:
# (텍스트, 임베딩) 쌍을 생성
text_embedding_pairs = list(zip(all_splits, embeddings_list))

# FAISS 벡터 스토어 생성
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,  # 올바른 형식 (튜플 리스트)
    embedding=embedding  # 임베딩 모델 객체
)

print("FAISS 벡터 저장 완료!")

In [ ]:
print("FAISS에 저장된 벡터 개수:", vector_store.index.ntotal)
print("실제 embeddings 개수:", len(embeddings_list))

# LoRA model load

In [9]:
#불러오기
FINETUNED_MODEL = base_path / "ko_gemma"
peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [10]:
# 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

In [ ]:
# Torch Dynamo 완전 비활성화 (강제)
torch._dynamo.config.suppress_errors = True
torch._inductor.config.fallback_random = True
torch._dynamo.reset()  # 기존 캐시 제거
torch._dynamo.config.cache_size_limit = 0  # 캐시 크기 제한
torch._dynamo.config.disable = True  # Dynamo 강제 비활성화

# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True
)

In [12]:
# LoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [ ]:
print("모델 타입:", type(peft_model))
print("토크나이저 타입:", type(tokenizer))

In [ ]:
# LoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

In [ ]:
print("모델 타입:", type(merged_model))
print("토크나이저 타입:", type(tokenizer))

In [16]:
# 경로 확인
test = pd.read_csv(base_path / 'test.csv', encoding = 'utf-8-sig') # 수정 해야함

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]
test['인적사고'] = test['인적사고'].str.replace(r'\(.*?\)', '', regex=True)


# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"{row['공종(중분류)']} 작업 중 {row['인적사고']} 발생. \n"
            f"키워드: {row['사고원인']} \n"
            f"{row['인적사고']} 방지를 위한 조치는?"
        )
    },
    axis=1
)
# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))
combined_test_data.head()

,question
0,철근콘크리트공사 작업 중 부딪힘 발생. \n키워드: 펌프카 아웃트리거 바닥 고임목을...
1,"수장공사 작업 중 절단, 베임 발생. \n키워드: 작업자의 불안전한 행동(숫돌 측면..."
2,미장공사 작업 중 떨어짐 발생. \n키워드: 작업자가 작업을 위해 이동 중 전방을 ...
3,조적공사 작업 중 넘어짐 발생. \n키워드: 작업 발판 위 벽돌 잔재를 밟고 넘어짐...
4,교량공사 작업 중 떨어짐 발생. \n키워드: 점심식사를 위한 이동시 작업자 부주의로...


In [ ]:
text_generation_pipeline = pipeline(
    model= merged_model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False, # False 로 하면 같은 입력에 같은 출력
    return_full_text=False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=8  # 배치 크기 지정
)

In [ ]:
print("모델 타입:", type(merged_model))
print("토크나이저 타입:", type(tokenizer))

In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [20]:
prompt_template = """
역할: 당신은 건설 안전 전문가입니다. 검색 결과를 바탕으로 질문에 대한 조치 사항을 간결하게 작성하세요.

질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
검색된 내용에 기반하여 조치를 작성하세요.
목차, 번호, 특수기호 없이 핵심 내용만 서술하세요.
답변에 불필요한 부연 설명, 연결어, 주어 제거하세요.
반드시 마침표로 끝나는 문장으로 작성하세요.
반드시 답변만 출력하세요.

{context}

질문:
{question}

답변:
"""


# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Result Generation

In [ ]:
# Cross-Encoder 모델 초기화 (한국어 모델 사용, max_length=512)
cross_encoder = CrossEncoder("bongsoo/albert-small-kor-cross-encoder-v1", max_length=512)
# TensorFloat32 경고 해결 (필요시)
torch.set_float32_matmul_precision('high')
tokenizer.padding_side = "left"  # tokenizer 객체가 있을 경우에만 사용

# 배치 크기를 줄여서 GPU 메모리 부담을 줄입니다.
batch_size = 8  # 예: 기존 16에서 4로 조정

test_questions = combined_test_data['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})
val_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

def process_batch(batch):
    batched_prompts = []
    retrieval_results = []
    for q in batch["question"]:
        # 1. Dense Retrieval으로 관련 문서(청크) 전체 가져오기
        dense_results = retriever.invoke(q)
        if dense_results:
            # 2. 각 (질문, 문서) 쌍에 대해 Cross-Encoder 점수 계산
            pairs = [(q, doc.page_content if hasattr(doc, "page_content") else doc) for doc in dense_results]
            ce_scores = cross_encoder.predict(pairs)
            # 3. 점수를 기준으로 내림차순 정렬 후 55% 이상인 문서 필터링
            reranked = sorted(zip(dense_results, ce_scores), key=lambda x: x[1], reverse=True)
            filtered_docs = [doc for doc, score in reranked if score >= 0.55]
            # 4. 조건에 만족하면 해당 문서들의 page_content를 결합하여 컨텍스트 생성, 아니면 빈 문자열
            if filtered_docs:
                context = "\n".join([doc.page_content for doc in filtered_docs])
            else:
                context = ""

            # 검색 결과 저장
            retrieval_results.append({
                "question": q,
                "retrieved_docs": [
                    {
                        "content": doc.page_content if hasattr(doc, "page_content") else doc,
                        "similarity_score": score
                    } for doc, score in reranked if score >= 0.55
                ]
            })
        else:
            context = ""
            retrieval_results.append({
                "question": q,
                "retrieved_docs": []
            })
        batched_prompts.append(prompt.format(context=context, question=q))

    # 모델 추론 시 기울기 계산 방지를 위해 no_grad 블록 사용
    with torch.no_grad():
        outputs = text_generation_pipeline(batched_prompts, batch_size=len(batched_prompts))
    # 각 출력 결과에서 'generated_text' 키의 값을 추출합니다.
    return [output[0]["generated_text"] for output in outputs], retrieval_results

# 메인 루프 수정 (배치 처리 후 캐시 클리어)
for batch in tqdm(test_dataset.batch(batch_size), desc="검증 배치 처리"):
    batch_results, batch_retrieval_results = process_batch(batch)
    val_results.extend(batch_results)
    # 배치 처리 후 GPU 캐시를 비워 메모리 누수를 방지합니다.
    torch.cuda.empty_cache()
print("\n테스트 처리 완료! 총 검증 결과 수:", len(val_results))

In [22]:
test_result = pd.DataFrame({
     "answer": val_results  # 모델이 생성한 답변 리스트
})
test_result

,answer
0,펌프카 아웃트리거 고임목 보강 및 지반 침하 방지 조치 실시. 좌우 아웃트리거 길이...
1,"작업자 안전교육 실시 및 안전수칙 준수 강조. \n보안면 착용 의무화. \n절단, ..."
2,작업 전 안전 교육 실시 및 작업자 안전 의식 고취.\n계단 주변 안전 표지판 설치...
3,작업 발판 위 잔재물 제거 및 발판 안전 점검 실시. 작업자 안전 교육 실시. 안전...
4,작업장 내 이동 시 안전벨트 착용 의무화 및 안전 교육 실시. 안전 난간 설치. 안...
5,안전발판 설치 의무화 및 작업절차 준수 교육 실시. 안전발판 설치 및 작업절차 준수...
6,안전 난간 설치 및 안전벨트 착용 의무화. 안전 교육 실시. \n안전 관리자 지정....
7,TSC GIRDER 조립 작업 시 SPLICE PLATE 설치 작업은 2인 이상 작...


# post_cleaning

In [23]:
def post_cleaning(df):
  # 1번. 1., 2. 등 제거
  df['answer'] = df['answer'].str.replace(r'^\d+\.', ',', regex=True)

  # 2번. 줄 띄움 -> , 으로 변경
  df['answer'] = df['answer'].str.replace('\n','', regex=False)

  # 3번. 질문부터 끝까지 삭제(있다면)
  df['answer'] = df['answer'].str.replace(r'질문.*', '', regex=True)

  # 4번. 연속된 ","를 "," 하나로 변경
  df['answer'] = df['answer'].str.replace(r',+', '.', regex=True)

  # 5번. 앞뒤 공백 및 "," 제거
  df['answer'] = df['answer'].str.strip()

  return df

In [24]:
test_result = post_cleaning(test_result)

# Submission Generation

In [28]:
from sentence_transformers import SentenceTransformer
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

In [29]:
# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_result['answer'])
print(pred_embeddings.shape)  # (샘플 개수, 768)

(8, 768)


In [ ]:
submission = pd.read_csv(base_path / 'sample_submission.csv', encoding = 'utf-8-sig') # 수정해야함
# 최종 결과 저장
submission.iloc[:,1] = test_result['answer']
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv(base_path / 'submission.csv', index=False, encoding='utf-8-sig') # 수정해야함